In [1]:
import msoffcrypto
import io
import glob
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path

# =========================
# INPUT DISINI

mo_folder = '3. Mar-23/'
data_date = '2023-03-31'

#  ========================================================================
#  TOOLS UNTUK REKAP FILE HARIAN KE LAPORAN BULANAN 'LIQUIDITY DASHBOARD'
#   by AL
#
#   FILE : 
#      (make sure nama Folder Bulan konsisten, eg."3. Mar-23")
#   1. Daily Report     : Monthly/Sumber Data/Daily Report
#   2. Nominatif Loan   : Monthly/Sumber Data/Nominatif Loan
#   3. Blotter Bulanan  : Monthly/Sumber Data/Blotter Bulanan
#   4. Nominatif Trea   : Monthly/Sumber Data/Nominatif Treasury
#   5. Liquidity Daily  : Daily Report/Nobu Liquidity Report
#
#   OUTPUT FILE : Monthly/Liquidity Dashboard/Output
#  ========================================================================

temp = io.BytesIO()

monthly_dir = 'D:/BCKP/Data C/Documents/NOBU OFFICE/REPORT/Monthly Report/'
sumber_dir = monthly_dir + 'Sumber Data/'
liq_db_dir = monthly_dir + 'Liquidity Dashboard/'

# DPK & LOAN ================================================
# source::
dailyrep_dir = sumber_dir + 'Daily Report/' + mo_folder

fh = pd.DataFrame()
loandpk = pd.DataFrame()

files = glob.glob(dailyrep_dir + '/*.xlsx')
for file in files :
    with open(file, 'rb') as f:
        excel = msoffcrypto.OfficeFile(f)
        excel.load_key('nobudaily')
        excel.decrypt(temp)
    
    print('reading > ' + Path(file).stem)
    
    #FH / contoh
    dailyrep_f = pd.read_excel(temp, sheet_name = "FH", skiprows = 5)
    new_fh = dailyrep_f.iloc[:,[2,5]]
    new_fh = new_fh.set_index('Financial Highlights')
    new_fh = new_fh.T
    new_fh['Date'] = new_fh.index
    fh = pd.concat([fh, new_fh], ignore_index = True)
    
    #LOAN & DPK
    daily_ld = pd.read_excel(temp, sheet_name = "Loan&DPK", skiprows = 4)
    new_loandpk = daily_ld.iloc[122:123,[1,8,9,10,11,12,13,7,6]]
    new_loandpk['Date'] = new_fh['Date'][0]
    loandpk = pd.concat([loandpk, new_loandpk], ignore_index = True)
    
loandpk['DPK IDR'] = loandpk['Giro'] + loandpk['Savings'] + loandpk['Deposito']
loandpk['DPK VA'] = loandpk['GIRO VALAS'] + loandpk['SAVING VALAS'] + loandpk['TD VALAS']

# NOMINATIF LOAN =====================================================
# Source ::
nomloan_dir = sumber_dir + 'Nominatif Loan/' + mo_folder

files = glob.glob(nomloan_dir + '/*.xlsb')
nomloan = pd.DataFrame()
for file in files :
    with open(file, 'rb') as f:
        excel = msoffcrypto.OfficeFile(f)
        excel.load_key('nobuloan')
        excel.decrypt(temp)
    
    print('reading > ' + Path(file).stem)
    
    nomloan_f = pd.read_excel(temp, sheet_name = "Financial", skiprows = 3)
    nomloan_smry = nomloan_f[nomloan_f['No'] == 'Overall - Summary']
    
    nomloan_f = nomloan_f.head(-46)
    #nomloan_smry.columns = range(nomloan_smry.columns.size)
    #nomloan_smry['filename'] = file[-15:-5]
    nomloan_smry['Date'] = datetime.strptime(file[-15:-5], '%d.%m.%Y')
    nomloan_smry['Unused Loan'] = nomloan_f['Kelonggaran Tarik'].sum()/1e6
    nomloan_smry['Unused PRK'] = nomloan_f[nomloan_f['Jenis Fasilitas'].str.startswith('PRK')]['Kelonggaran Tarik'].sum()/1e6 
    
    nomloan = pd.concat([nomloan, nomloan_smry[['Date', 'Unused Loan', 'Unused PRK']]], ignore_index = True)


# Taking Bulanan (BLOTTER) =================================================
# source ::
blotter_dir = sumber_dir + 'Blotter Bulanan/' + mo_folder

files = glob.glob(blotter_dir + '/*.xls')
blotter = pd.DataFrame()

for file in files :
    blotter_f = pd.read_excel(file, skiprows = 6)

blotter = blotter_f[(blotter_f['C\'\'Party Type'] == 'InterBank') & (blotter_f['Buy/Sell'] == 'Taking')]
blotter['Trade Date edit'] = pd.to_datetime(blotter.loc[:,'Trade Date'], format='%d/%m/%Y')
blotter_group = blotter.groupby(['Trade Date edit','Other Ccy'])['Trade Amount'].sum().unstack(level = 1)
blotter_group['Date'] = blotter_group.index
blotter_group['IDR'] = blotter_group['IDR']/1e6
blotter_group['USD'] = blotter_group['USD']/1e3

liq_group = pd.merge(loandpk, nomloan, on ='Date', how= 'left')
liq_group = pd.merge(liq_group, blotter_group, on ='Date', how= 'left')
#Liq Group -> OUT
    
    
# Daily Nominative Treasury Report (NON BI Taking) =============================================
# source::

nomtre_dir = sumber_dir + 'Nominatif Treasury/' + mo_folder
nonbi = pd.DataFrame()
files = glob.glob(nomtre_dir + '/*.xls')

for file in files :
    nomtre_f = pd.read_excel(file)
    nomtre_date = nomtre_f.iloc[0,0]
    nomtre_f = pd.read_excel(file, skiprows=3)

    print('reading > ' + Path(file).stem)
    
    #INDEXING
    i_nonbi_idr_top, i_nonbi_idr_btm = nomtre_f[nomtre_f['Ticket'] == 'Group Type : Non BI'].index[0], nomtre_f[nomtre_f['Ticket'] == 'Total : Non BI Taking'].index[-2] 
    i_nonbi_usd_top, i_nonbi_usd_btm = nomtre_f[nomtre_f['Ticket'] == 'Group Type : Non BI'].index[-1], nomtre_f[nomtre_f['Ticket'] == 'Total : Non BI Taking'].index[-1]
    
    nonbi_idr, nonbi_usd = nomtre_f.iloc[i_nonbi_idr_top : i_nonbi_idr_btm] , nomtre_f.iloc[i_nonbi_usd_top : i_nonbi_usd_btm]
    
    nonbi_idr['Date'] = datetime.strptime(nomtre_date[-10:], '%d/%m/%Y')
    nonbi_idr['Currency'] = 'IDR'
    nonbi_usd['Date'] = datetime.strptime(nomtre_date[-10:], '%d/%m/%Y')
    nonbi_usd['Currency'] = 'USD'

    nonbi = pd.concat([nonbi, nonbi_idr, nonbi_usd], ignore_index = True)

nonbi = nonbi[~nonbi['Ticket'].isna()]
nonbi = nonbi[nonbi['Nominal'] < 0]

mask_mm   = nonbi['Ticket'].str.startswith('MM')
nonbi.loc[mask_mm, 'MM/REPO']   = 'MM'
mask_repo = nonbi['Ticket'].str.startswith('REPO')
nonbi.loc[mask_repo, 'MM/REPO'] = 'REPO'

nonbi['Key'] = nonbi['Currency'] + '-' + nonbi['MM/REPO'] 

reading > Daily Report 01.03.2023
reading > Daily Report 02.03.2023
reading > Daily Report 03.03.2023
reading > Daily Report 06.03.2023
reading > Daily Report 07.03.2023
reading > Daily Report 08.03.2023
reading > Daily Report 09.03.2023
reading > Daily Report 10.03.2023
reading > Daily Report 13.03.2023
reading > Daily Report 14.03.2023
reading > Nominatif Loan 01.03.2023
reading > Nominatif Loan 02.03.2023
reading > Nominatif Loan 03.03.2023
reading > Nominatif Loan 06.03.2023
reading > Nominatif Loan 08.03.2023
reading > Nominatif Loan 09.03.2023
reading > Nominatif Loan 10.03.2023
reading > Nominatif Loan 13.03.2023
reading > Nominatif Loan 14.03.2023
reading > Nominatif Loan 28.02.2023


C:\Users\Asus\AppData\Local\Temp\ipykernel_10160\733294384.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blotter['Trade Date edit'] = pd.to_datetime(blotter.loc[:,'Trade Date'], format='%d/%m/%Y')


reading > Nobu Liquidity Risk Report - 01032023
reading > Nobu Liquidity Risk Report - 02032023
reading > Nobu Liquidity Risk Report - 03032023
reading > Nobu Liquidity Risk Report - 06032023
reading > Nobu Liquidity Risk Report - 07032023
reading > Nobu Liquidity Risk Report - 08032023
reading > Nobu Liquidity Risk Report - 09032023
reading > Nobu Liquidity Risk Report - 10032023
reading > Nobu Liquidity Risk Report - 13032023
reading > Nobu Liquidity Risk Report - 14032023
reading > Nobu Liquidity Risk Report - 15032023
reading > Nobu Liquidity Risk Report - 16032023
reading > Nobu Liquidity Risk Report - 17032023
reading > Nominatif 01 Maret 2023
reading > Nominatif 02 Maret 2023
reading > Nominatif 03 Maret 2023


C:\Users\Asus\AppData\Local\Temp\ipykernel_10160\733294384.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonbi_idr['Date'] = datetime.strptime(nomtre_date[-10:], '%d/%m/%Y')
C:\Users\Asus\AppData\Local\Temp\ipykernel_10160\733294384.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonbi_idr['Currency'] = 'IDR'
C:\Users\Asus\AppData\Local\Temp\ipykernel_10160\733294384.py:201: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [ ]:
# Daily MLR Liquidity Report =============================================
import openpyxl
import warnings
warnings.filterwarnings('ignore', category=UserWarning,module='openpyxl')
# Source ::
dailymlr_dir = 'D:/BCKP/Data C/Documents/NOBU OFFICE/REPORT/Daily Report/'
dailyliq_dir = dailymlr_dir + 'Nobu Liquidity Report/'

sh_al = pd.DataFrame()
sh_ews = pd.DataFrame()

files = glob.glob(dailyliq_dir + mo_folder + '/*.xlsm')

for file in files :
    wb = openpyxl.load_workbook(file, data_only = True)
    print('reading > ' + Path(file).stem)
    
    sh1 = wb['Monitoring Liquidity Risk']
    sh2 = wb['EWS Indicator']

    al_date    = sh1['F10'].value
    al_alikuid = sh1['F27'].value
    al_tdpk    = sh1['F29'].value
    al_tncd    = sh1['F28'].value
    al_alncd   = sh1['F11'].value
    al_aldpk   = sh1['F12'].value

    sh_al_new = {'Date'     : al_date, 
                 'A likuid' : al_alikuid,
                 'Tot DPK'  : al_tdpk,
                 'Tot NCD'  : al_tncd,
                 'AL/NCD'   : al_alncd,
                 'AL/DPK'   : al_aldpk
                }
    sh_al_new  = pd.DataFrame(sh_al_new, index = [0])
    sh_al      = pd.concat([sh_al, sh_al_new], ignore_index = True)
    #sh_al -> Out
    
    ews_rim       = sh2['G5'].value
    ews_gwmidr_ha = sh2['G8'].value
    ews_gwmidr_rt = sh2['G9'].value
    ews_gwmva_ha  = sh2['G10'].value
    ews_gwmva_rt  = sh2['G11'].value
    ews_plm       = sh2['G12'].value
    ews_inti      = sh2['G13'].value

    sh_ews_new = {'Date'       : al_date, 
                  'RIM'        : ews_rim,
                  'ALNCD'      : al_alncd,
                  'ALDPK'      : al_aldpk,
                  'GWM idr_ha' : ews_gwmidr_ha,
                  'GWM idr_rt' : ews_gwmidr_rt,
                  'GWM va_ha'  : ews_gwmva_ha,
                  'GWM va_rt'  : ews_gwmva_rt,
                  'PLM'        : ews_plm,
                  'Dep Inti'   : ews_inti
                }
    sh_ews_new  = pd.DataFrame(sh_ews_new, index = [0])
    sh_ews      = pd.concat([sh_ews, sh_ews_new], ignore_index = True)
    #sh_ews -> Out

**OUT EXPORT**

In [2]:
writer = pd.ExcelWriter(liq_db_dir + 'Output/Liquidity Dashboard ' + data_date +' Output.xlsx', engine = 'xlsxwriter')

liq_group[['Date','Giro','Savings', 'Deposito', 'GIRO VALAS', 'SAVING VALAS', 'TD VALAS', 
           'DPK IDR', 'DPK VA', 'Loans', 'Unused Loan', 'Unused PRK', 'IDR', 'USD']].to_excel(writer, sheet_name = 'Liq Dashb DPK Loan', index = False)
sh_al.to_excel(writer, sheet_name = 'ALNCD ALDPK', index = False)
sh_ews.to_excel(writer, sheet_name = 'EWS', index = False)
nonbi.groupby(['Date','Key'])['Nominal'].sum().unstack(level=1).to_excel(writer, sheet_name = 'Nominatif Treasury', index = False)

writer.close()